<a href="https://colab.research.google.com/github/ssunggun2/Portfolio/blob/main/juso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install transformers
!pip install ratsnlp

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# NER MODEL 

In [38]:
import os
import re
import torch
from transformers import ElectraConfig, ElectraForTokenClassification, ElectraTokenizer
# import pandas as pd

# pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
# pd.set_option('display.max_rows', None) ## 모든 열을 출력한다.
# pd.set_option('display.max_colwidth', -1)

from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name="monologg/koelectra-base-v3-generator",
    downstream_model_dir="/content/drive/MyDrive/aimpact/220127",
    downstream_model_labelmap_fpath="/content/drive/MyDrive/aimpact/220127/label_map.txt",
    max_seq_length = 512,
)

fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

pretrained_model_config = ElectraConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = ElectraForTokenClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()}, strict=False)           #loading state_dict for ... : Missing key(s) in state_dict:... 오류 해결 (strict=False 추가)
model.eval()

tokenizer = ElectraTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  id_to_label[idx] = label  
  
def inference_fn(sentence):

    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        probs = outputs[0][0].softmax(dim=1)
        top_probs, preds = torch.topk(probs, dim=1, k=1)
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        predicted_tags = [id_to_label[pred.item()] for pred in preds]
        result = []
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result = {
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)

    return {
        "sentence": sentence,
        "result": result,
    }

downstream_model_checkpoint_fpath: /content/drive/MyDrive/aimpact/220127/epoch=2-val_loss=0.18.ckpt
downstream_model_labelmap_fpath: /content/drive/MyDrive/aimpact/220127/label_map.txt


# 문장  태그 분석 전에서 전화번호 이름 추출 해야한다?

## 전화번호 너구리로 추출 + 분석할 수 있도록 원본 메시지와 같은 idx 유지하는 메시지 return

In [39]:
# 너구리 전화번호 추출
def getphonelist(msg):
    msg_list = []
    num_start_list = []
    num_end_list = []
    num_length_list = []

    regex = re.compile(r"0(?:10|11|13[0-9]|16|17|18|19|2|31|32|33|41|42|43|44|50[1-9]?|51|52|53|54|55|61|62|63|64|70)[ ]*(?:[^\w\s]|ㅡ|ㆍ){0,2}[ ]*[0-9]{3,4}[ ]*(?:[^\w\s]|ㅡ|ㆍ){0,2}[ ]*[0-9]{4}")
    result = re.findall(regex, msg)
    result_iter = re.finditer(regex, msg)

    for r in result_iter:
        num_start = r.start()
        num_end = r.end()
        num_length = num_end - num_start
        num_start_list.append(num_start)
        num_end_list.append(num_end)
        num_length_list.append(num_length)
    
    if len(num_start_list) != 0:
        target_msg = msg
        
        for i in range(len(num_start_list)):
            
            num_start_idx = num_start_list[i]
            num_end_idx = num_end_list[i]
            
            tmp = target_msg[:num_start_idx]
            tmp2 = target_msg[num_end_idx:]

            target_msg = tmp + ' '*(num_length_list[i]) + tmp2
        
        # target_msg를 공백(전화번호를 치환한) 단위로 나누기
        unlength_list = target_msg.split("         ")  # 빈칸 9개
        msg_list.append(unlength_list[0])
        for i in range(1,len(unlength_list)):
            i_msg = unlength_list[i]
            i_msg = "         " + i_msg     # 빈칸 9개
            msg_list.append(i_msg)

    else:
        msg_list.append(msg)

    # for phoneIndex in range(len(result)):
    #     result[phoneIndex] = result[phoneIndex].replace(" ", "")
    #     result[phoneIndex] = result[phoneIndex].replace("-", "")
    #     result[phoneIndex] = result[phoneIndex].replace("~", "")
    #     result[phoneIndex] = result[phoneIndex].replace("ㅡ", "")
    #     result[phoneIndex] = result[phoneIndex].replace(".", "")


    return result, msg_list, msg

In [40]:
def preprocessing(msg_list):            # input = list / output = list
    #

    regex = re.compile(r" \d+\. | ?연락처\d+\.? ?|\n\d+\. ")        ######### 추가 필요
    # print(f"msg_list : {msg_list}")
    # print(f"len_msg_list : {len(msg_list)}")
    prepro_msg_list = []

    for msg in msg_list:

        stop_start_list = []
        stop_end_list = []
        stop_length_list = []
        # print(f"msg : {msg}")
        
        result_iter = re.finditer(regex, msg)
        # print(f"result_iter : {result_iter}")
        
        for r in result_iter:
            # print(f"r : {r}")
            stop_start = r.start()
            stop_end = r.end()
            stop_length = stop_end - stop_start
            stop_start_list.append(stop_start)
            stop_end_list.append(stop_end)
            stop_length_list.append(stop_length)
        
        # print(f"stop_start_list : {stop_start_list}")
        # print(f"stop_end_list : {stop_end_list}")
        
        if len(stop_start_list) != 0:
            target_msg = msg
            
            # print(f"len_ori_msg : {len(msg)}")
            # print(f"ori_msg : {msg}")

            for i in range(len(stop_start_list)):
                
                stop_start_idx = stop_start_list[i]
                stop_end_idx = stop_end_list[i]
                
                tmp = target_msg[:stop_start_idx]
                tmp2 = target_msg[stop_end_idx:]

                target_msg = tmp + ' '*(stop_length_list[i]) + tmp2
                
            prepro_msg_list.append(target_msg)
                # print(f"len_prepor_msg : {len(target_msg)}")
                # print(f"prepor_msg : {target_msg}")

        else:
            prepro_msg_list.append(msg)
            # print(f"len_prepor_msg : {len(prepro_msg_list)}")
            # print(f"prepor_msg : {prepro_msg_list}")

        # prepro_msg = re.sub(regex, " ", msg)
        # prepro_msg_list.append(prepro_msg)

    return prepro_msg_list

# 분석

# 문장 태깅

In [41]:
# @ 문자 치환
def replace_str(sentence):
    if '@' in sentence:
        sentence = sentence.replace("@", "아파트")
    else:
        sentence=sentence

    return sentence

# 태그 다는 함수
def get_output(sentence):

    results = []
    outputs = []
    sent = {"sentence" : sentence}
    outputs.append(sent)

    # 개행 여부에 따라서 input list 만들기
    if '\n' in sentence:                        # 개행 있으면 리스트 안에 개행된 문장으로 나눠져서 차례차례 들어감
        aligned_sentence = sentence.split('\n')
        
    else:                                       # 개행 없으면 리스트 안에 변수 한개 밖에 없음
        aligned_sentence = []
        aligned_sentence.append(sentence)
    

    for sent in aligned_sentence:
        sent = replace_str(sent)
        result = inference_fn(sent)
        words = result["sentence"]
        resultArray = result["result"]
        currentToken = ""
        currentTag = ""
        baseIdx = 0
        tokenResults = []
        for i in range(0, len(resultArray)):
            tokenInfo = resultArray[i]
            token = tokenInfo["token"].strip("#")
            predictedTag = tokenInfo["predicted_tag"]
            if (predictedTag.startswith("I-")) :
                for j in range(0, 10):
                    blankStr = ""
                    for k in range(1, j):
                        blankStr += " "
                    tempToken = currentToken + blankStr + token
                    if (words.find(tempToken, baseIdx) != -1):
                        currentToken = tempToken
                        break
                continue

            if (predictedTag == "O") :
                if (currentToken != "") :
                    begin = words.find(currentToken, baseIdx)
                    end = begin + len(currentToken)
                    baseIdx = end


                    element = {
                        "id": len(tokenResults) + 1,
                        "form": currentToken,
                        "label": currentTag[2:],
                        "begin": begin,
                        "end": end
                    }
                    tokenResults.append(element)
                    currentToken = ""
                    currentTag = ""
                continue

            if (currentToken != "") :
                begin = words.find(currentToken, baseIdx)
                end = begin + len(currentToken)
                baseIdx = end

                element = {
                    "id": len(tokenResults) + 1,
                    "form": currentToken,
                    "label": currentTag[2:],
                    "begin": begin,
                    "end": end
                }
                tokenResults.append(element)
            
            currentToken = token
            currentTag = predictedTag

        if (currentToken != "") :
            begin = words.find(currentToken, baseIdx)
            end = begin + len(currentToken)
            baseIdx = end

            element = {
                "id": len(tokenResults) + 1,
                "form": currentToken,
                "label": currentTag[2:],
                "begin": begin,
                "end": end
            }
            tokenResults.append(element)
        
        results.append(tokenResults)

        output = {
            "form": words,
            "ne": tokenResults
        }

        outputs.append(output)

    return outputs


# 주소추출 + 주소부분 특수문자 처리

In [42]:
# 주소 시작 부분 패턴
def new_get_start_patten(tags):

    start_list = []
    start_idx = 0

    # 3. 주소의 시작으로 보는 패턴이 있는지 확인한다.
    for i in range(len(tags)):

        # 마지막 전 태그와 마지막 태그 인 경우 무시
        if i != (len(tags) -1) and i != len(tags):

            if tags[i] == 'LCP_CAPITALCITY' and (tags[i+1] == 'LCP_COUNTY' or tags[i+1] == 'AF_ROAD' or tags[i+1] == 'QT_ADDRESS' or tags[i+1] == 'LC_OTHERS' or tags[i+1] == 'AF_BUILDING' or tags[i+1] == 'LC_OTHERS'):
# LC_OTHERS,              
                start_idx = i
                start_list.append(start_idx)

            elif tags[i] == 'LCP_PROVINCE' and (tags[i+1] == 'LCP_COUNTY' or tags[i+1] == 'AF_ROAD' or tags[i+1] == 'LCP_CITY' or tags[i+1] == 'QT_ADDRESS' or tags[i+1] == 'AF_TRANSPORT' or tags[i+1] == 'AF_BUILDING' or tags[i+1] == 'QT_ORDER' or tags[i+1] == 'LC_OTHERS'):
# AF_BUILDING, QT_ORDER, LC_OTHERS
                start_idx = i
                start_list.append(start_idx)

            elif tags[i] == 'LCP_CITY' and (tags[i+1] == 'LCP_COUNTY' or tags[i+1] == 'LCP_CITY' or tags[i+1] == 'LC_OTHERS' or tags[i+1] == 'LCG_MOUNTAIN' or tags[i+1] == 'LCG_ISLAND' or tags[i+1] == 'AF_ROAD' or tags[i+1] == 'AF_BUILDING' or tags[i+1] == 'QT_ADDRESS' or tags[i+1] == 'QT_OTHERS' or tags[i+1] == 'PT_TREE' or tags[i+1] == 'TMI_MODEL' or tags[i+1] == '' or tags[i+1] == 'AM_BIRD' or tags[i+1] == 'PS_NAME' or tags[i+1] == 'QT_ORDER'):
# TMI_MODEL, '', AM_BIRD, PS_NAME, QT_ORDER
                start_idx = i
                start_list.append(start_idx)
                ## CITY 뒤에 빌딩 빼야하나??

# LCP_COUNTY 전부 새로 추가
            elif tags[i] == 'LCP_COUNTY' and (tags[i+1] == 'LCP_COUNTY' or tags[i+1] == 'AF_ROAD' or tags[i+1] == 'QT_ADDRESS' or tags[i+1] == 'LC_OTHERS' or tags[i+1] == 'AF_BUILDING' or tags[i+1] == 'LCP_CITY' or tags[i+1] == 'AFA_PERFORMANCE' or ('OGG' in tags[i+1])):
                start_idx = i
                start_list.append(start_idx)

# AF_ROAD 전부 새로 추가
            elif tags[i] == 'AF_ROAD' and (tags[i+1] == 'QT_ADDRESS' or tags[i+1] == 'LCP_COUNTY' or tags[i+1] == 'AF_ROAD' or tags[i+1] == 'QT_ORDER'):
                start_idx = i
                start_list.append(start_idx)

# AF_BUILDING 전부 새로 추가
            elif tags[i] == 'AF_BUILDING' and (tags[i+1] == 'QT_ADDRESS' or tags[i+1] == 'QT_OTHERS' or tags[i+1] == 'LCP_COUNTY' or tags[i+1] == 'AF_BUILDING' or tags[i+1] == 'QT_ORDER' or tags[i+1] == 'AF_ROAD'):
                start_idx = i
                start_list.append(start_idx)

# OGG 전부 새로 추가
            elif ('OGG' in tags[i]) and (tags[i+1] == 'LCP_COUNTY' or tags[i+1] == 'QT_ORDER'):
                start_idx = i
                start_list.append(start_idx)

# CV_POSITION 전부 새로 추가
            elif tags[i] == 'CV_POSITION' and (tags[i+1] == 'LCP_COUNTY'):
                start_idx = i
                start_list.append(start_idx)
# LC_OTHERS
            elif tags[i] == 'LC_OTHERS' and (tags[i+1] == 'LC_OTHERS' or tags[i+1] == 'QT_ADDRESS'):
                start_idx = i
                start_list.append(start_idx)

# LCG_ISLAND
            elif tags[i] == 'LCG_ISLAND' and (tags[i+1] == 'LCP_COUNTY'):
                start_idx = i
                start_list.append(start_idx)


            else:
                continue

        else:
            break

    # print(f"뭐지?: {start_list}")

    removed_list = []
    if len(start_list) >=2:
       
        removed_list.append(start_list[0])


        # 10589 번 경기 안잡히는 케이스 관련 
        for j in range(1, len(start_list)):
            # print(f"뭐지3?: {start_list[j-1], start_list[j]}")
            if start_list[j-1] == start_list[j] - 1:    
                # pass                                    ################## 이게 맞는듯
                if j == 1:
                    pass
                else:
                    removed_list.append(start_list[j-1])
                    # print(f"뭐지2?: {removed_list}")

            else:
                removed_list.append(start_list[j])
                # print(f"뭐지2?: {removed_list}")

    else:
        removed_list = start_list
        # print(f"뭐지2?: {removed_list}")
    
    return removed_list

In [43]:
# 주소 끝부분 패턴 
def get_end_patten(tags):

    # 전화번호 형태 정규표현식
    regex_phone = re.compile('\d{2,4}-?.?\s{,3}\d{3,4}-?.?\s{,3}\d{4}')

    end_idx = 0
    idx_list = []
    idx_list.append(end_idx)
    # 3. 주소의 끝으로 보는 패턴이 있는지 확인한다.
    for i in range(len(tags)-2):

        if tags[i] == 'LCP_CAPITALCITY':
            if tags[i+1] == 'LCP_COUNTY':
                if tags[i+2] == 'AF_ROAD':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
        
        elif tags[i] == 'LCP_CITY':
            if tags[i+1] == 'LCP_COUNTY':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'AF_ROAD':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'AF_ROAD':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
        
        elif tags[i] == 'LCP_COUNTY':
            if tags[i+1] == 'AF_ROAD':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'LCP_COUNTY':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'AF_BUILDING':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'LCP_COUNTY':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'AF_BUILDING':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'QT_ADDRESS' and (regex_phone.search(tags[i+1]) == None):
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'AF_BUILDING':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if 'OGG' in tags[i+2]:
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'QT_ORDER':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
                
        elif tags[i] == 'AF_ROAD':
            if tags[i+1] == 'QT_ADDRESS' and (regex_phone.search(tags[i+1]) == None):
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'AF_BUILDING':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
                
                if tags[i+2] == 'LCP_COUNTY':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
                
                if 'OGG' in tags[i+2]:
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_OTHERS':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'AF_ROAD':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
           
            if tags[i+1] == 'AF_BUILDING':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
            
            if tags[i+1] == 'QT_ORDER':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
                
                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
            
            if tags[i+1] == 'AF_ROAD':
                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
        
        elif tags[i] == 'AF_BUILDING':
            if tags[i+1] == 'QT_ADDRESS' and (regex_phone.search(tags[i+1]) == None):
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'AF_BUILDING':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'DT_YEAR':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'QT_ORDER':
                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_OTHERS':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass


            if tags[i+1] == 'AF_BUILDING':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass


            if tags[i+1] == 'LCP_COUNTY':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'QT_OTHERS':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

        elif tags[i] == 'QT_ADDRESS' and (regex_phone.search(tags[i]) == None):
            if tags[i+1] == 'QT_ADDRESS' and (regex_phone.search(tags[i+1]) == None):
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'LCP_COUNTY':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'AF_BUILDING':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
                    
            if tags[i+1] == 'QT_ORDER':
                if 'OGG' in tags[i+2]:
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
                
                if tags[i+2] == 'AF_BUILDING':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'AF_BUILDING':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'AF_ROAD':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'LCP_COUNTY':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'AF_BUILDING':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
                
                if 'OGG' in tags[i+2]:
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if 'OGG' in tags[i+1]:
                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass     
                    
        
        elif tags[i] == 'QT_ORDER':
            if tags[i+1] == 'AF_ROAD':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'QT_ADDRESS' and (regex_phone.search(tags[i+1]) == None):
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
                
                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_OTHERS':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'AF_BUILDING':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

                if tags[i+2] == 'QT_ORDER':
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

            if tags[i+1] == 'QT_ORDER':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass
            
            if tags[i+1] == 'LCP_COUNTY':
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

        elif tags[i] == 'LC_OTHERS':
            if tags[i+1] == 'QT_ADDRESS' and (regex_phone.search(tags[i+1]) == None):
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

        elif 'OGG' in tags[i]:
            if tags[i+1] == 'QT_ADDRESS' and (regex_phone.search(tags[i+1]) == None):
                if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
                    end_idx = i+2
                    idx_list.append(end_idx)
                    pass

        # elif tags[i] == '':
        #     if tags[i+1] == 'QT_ADDRESS' and (regex_phone.search(tags[i+1]) == None):
        #         if tags[i+2] == 'QT_ADDRESS' and (regex_phone.search(tags[i+2]) == None):
        #             end_idx = i+2
        #             idx_list.append(end_idx)
        #             pass
        
        else:
            continue
    end_idx = max(idx_list)

    return end_idx


In [44]:
###### 상세주소 추출 만들다 말았음

# 주소 리턴하는 방식
def get_juso_idx(taglist, start_list):  
    
    taglist = taglist
    start_list = start_list

    start_idx_list = []
    end_idx_list = []

    # print(f"taglist : {taglist}")    
    # print(f"start_list : {start_list}")    
    for i in range(len(start_list)):
        start_idx = start_list[i]
        start_idx_list.append(start_idx)
        
        if i != len(start_list) - 1:
            next_start_idx = start_list[i+1]
            end_idx = get_end_patten(taglist[start_idx:next_start_idx])              # input = list / output = int
            end_idx_list.append(end_idx)
        else:
            end_idx = get_end_patten(taglist[start_idx:])              # input = list / output = int
            end_idx_list.append(end_idx)

    print(f"start_idx_list : {start_idx_list}")
    print(f"end_idx_list : {end_idx_list}")

    return start_idx_list, end_idx_list



In [45]:
def new_get_juso(msg_list, raw_msg):

    #### @ => 아파트 변환 과정에서 생기는 index오류 수정 필요
    # else: 문장에 @ 안들어갔으니까 아래 코드 그대로 진행

    print(f"raw_msg : {raw_msg}") 
    # print(f"len_raw_msg : {len(raw_msg)}")

    jusos = []
    replace_msg = []
    
    # print(f"len_msg_list : {len(msg_list)}")
    
    offset = 0
    if len(msg_list) != 1:
        
        for msg in msg_list:
            # print(f"offset : {offset}")
            msg = msg.replace("\n"," ").replace('.', ' ').replace('(', ' ').replace(')', ' ').replace('[', ' ').replace(']', ' ')
            outputs = get_output(msg)     # input = str / output = list

            for i in range(1,len(outputs)):
                
                output = outputs[i]
                form_list = []
                tag_list = []
                begin_list = []
                end_list = []
                nes = output['ne']

                for ne in nes:
                    form  = ne['form']
                    label = ne['label']
                    begin = ne['begin']
                    end = ne['end']
                    
                    form_list.append(form)
                    tag_list.append(label)
                    begin_list.append(begin)
                    end_list.append(end)
                print(f"msg : {msg}")
                print(f"form_list : {form_list}")
                print(f"tag_list : {tag_list}")
                # print(f"begin_list : {begin_list}")
                # print(f"end_list : {end_list}")


                start_idx_list = new_get_start_patten(tag_list)          # input = list  /  output = list
                if len(start_idx_list) == 0:
                    pass
                
                else:
                    start_idx_list, end_idx_list = get_juso_idx(tag_list, start_idx_list)           # input = tag_list, start_idx_list / output = int, int

                    target_msg = msg

                    for z in range(len(start_idx_list)):

                        start_idx = start_idx_list[z]
                        end_idx = end_idx_list[z]

                        # end_idx가 0이면 주소로 안찍히게
                        if end_idx == 0:
                            continue
                        
                        else:

                            char_begin = begin_list[start_idx]
                            char_end = end_list[start_idx + end_idx]
                            msg_juso = raw_msg[char_begin + offset : char_end + offset]
                            # print(f"char_end : {char_end}")
                            # print(f"len_msg : {len(msg)}")
                            # print(f"len_msg_juso : {len(msg_juso)}")
                            print(f"msg_juso : {msg_juso}")

                            if char_end >= len(msg):
                                msg_juso = raw_msg[char_begin + offset : char_end + offset]
                                pass
                            
                            else:
                                # print(f"char_end + offset : {char_end + offset}")
                                # print(f"raw_msg[char_end + offset] : {raw_msg[char_end + offset]}")
                                if raw_msg[char_end + offset] ==  ')':
                                    # print("YES!!! THERE ARE ')'")
                                    msg_juso = raw_msg[char_begin + offset : char_end + 1 + offset]
                                else:
                                    # print("NO!!! THERE ARE NO ')'")
                                    msg_juso = raw_msg[char_begin + offset : char_end + offset]

                            # @ => 아파트 변환 과정에서 생기는 index오류 수정 필요
                            if "@" in msg_juso:
                                char_end = char_end - 2
                                msg_juso = raw_msg[char_begin + offset : char_end + offset]
                            
                            else:
                                pass

                            juso = {
                                'address' : msg_juso,                   #기본 주소 (일단 주소 전체?) # str
                                'detailAddress' : msg_juso,           # 상세 주소 # str
                                'addrRocation' : msg_juso              # str 원본 메시지에서의 주소 부분
                            }
                            jusos.append(juso)

                            tmp = target_msg[:char_begin]
                            tmp2 = target_msg[char_end:]

                            long = len(msg_juso)
                            target_msg = tmp + ' '*(long) + tmp2

                            replace_msg.append(target_msg)

            # print(f"len_msg : {len(msg)}")
            offset += len(msg)
    
    else:

        for msg in msg_list:
            
            msg = msg.replace("\n"," ").replace('.', ' ').replace('(', ' ').replace(')', ' ').replace('[', ' ').replace(']', ' ')
            outputs = get_output(msg)     # input = str / output = list
            target_msg = msg

            for i in range(1,len(outputs)):
                
                output = outputs[i]
                form_list = []
                tag_list = []
                begin_list = []
                end_list = []
                nes = output['ne']

                for ne in nes:
                    form  = ne['form']
                    label = ne['label']
                    begin = ne['begin']
                    end = ne['end']
                    
                    form_list.append(form)
                    tag_list.append(label)
                    begin_list.append(begin)
                    end_list.append(end)
                print(f"msg : {msg}")
                print(f"form_list : {form_list}")
                print(f"tag_list : {tag_list}")
                # print(f"begin_list : {begin_list}")
                # print(f"end_list : {end_list}")

                start_idx_list = new_get_start_patten(tag_list)          # input = list  /  output = list
                if len(start_idx_list) == 0:
                    pass
                
                else:
                    start_idx_list, end_idx_list = get_juso_idx(tag_list, start_idx_list)           # input = tag_list, start_idx_list / output = int, int
                    
                    for z in range(len(start_idx_list)):

                        start_idx = start_idx_list[z]
                        end_idx = end_idx_list[z]
                        
                        # end_idx가 0이면 주소로 안찍히게
                        if end_idx == 0:
                            continue
                        
                        else:
                            char_begin = begin_list[start_idx]
                            char_end = end_list[start_idx + end_idx]
                            msg_juso = raw_msg[char_begin + offset : char_end + offset]
                            # print(f"char_end : {char_end}")
                            # print(f"len_msg : {len(msg)}")
                            # print(f"len_msg_juso : {len(msg_juso)}")
                            print(f"msg_juso : {msg_juso}")
                            
                            if char_end >= len(msg):
                                msg_juso = raw_msg[char_begin + offset : char_end + offset]
                                pass
                            
                            else:
                                # print(f"char_end + offset : {char_end + offset}")
                                # print(f"raw_msg[char_end + offset] : {raw_msg[char_end + offset]}")
                                if raw_msg[char_end + offset] ==  ')':
                                    # print("YES!!! THERE ARE ')'")
                                    msg_juso = raw_msg[char_begin + offset : char_end + 1 + offset]
                                else:
                                    # print("NO!!! THERE ARE NO ')'")
                                    msg_juso = raw_msg[char_begin + offset : char_end + offset]
                            
                            # @ => 아파트 변환 과정에서 생기는 index오류 수정 필요
                            if "@" in msg_juso:
                                char_end = char_end - 2
                                msg_juso = raw_msg[char_begin + offset : char_end + offset]
                            
                            else:
                                pass

                            juso = {
                                'address' : msg_juso,                   #기본 주소 (일단 주소 전체?) # str
                                'detailAddress' : msg_juso,           # 상세 주소 # str
                                'addrRocation' : msg_juso              # str 원본 메시지에서의 주소 부분
                            }
                            jusos.append(juso)


                            tmp = target_msg[:char_begin]
                            tmp2 = target_msg[char_end:]

                            long = len(msg_juso)
                            target_msg = tmp + ' '*(long) + tmp2

        replace_msg.append(target_msg)

    return jusos, replace_msg

# 이름 추출 (너구리 + 태그 분석)

In [46]:
# 이름을 분리하기위해 제거되어야하는 불용어리스트
stopwordsForName = [
    "대장길",  "수령지",  "마을",  "아파트",  "이사장",  "한의원",  "경기도",  "전남",  "우리집",  "한박스",  "양구",  "유선상",  "한라봉",  "천혜양",  "천혜향",  "배송비",  "배송지",  "이킬로",  "이키로",  "오킬로",  "오키로",  "주세요",  "한의원",  "현금영수증",  "송도문화로",  "구매자",  "판매자",  "입금자",  "신세계",  "소나무",  "문앞에",  "나눠서",  "주시면",  "주문서",  "금액은",  "우엉차",  "이체한",  "강사님",  "구분만",  "마늘도",  "여기도",  "편안한",  "한다고",  "마상품",  "한병",  "여주즙",  "소포장",  "여기로",  "하느라",  "위와",  "이름은",  "금욜",  "문자로",  "한다",  "안함",  "여름만",  "이쁜",  "신주소",  "나무문",  "성내동",  "주시고",  "백도",  "황도",  "배송전",  "배송후",  "조끔찍",  "오후에",  "오전에",  "현영",  "문앞",  "주소지",  "제주시",  "천곡동",  "한상자",  "주소가",  "지켜봐",  "우리의",  "이름이",  "최선을",  "심의필",  "이것도",  "표면이",  "문의를",  "한달이",  "문제가",  "도착할",  "어제거",  "선물로",  "배송시",  "표를",  "선물용",  "이번주",  "편의점",  "고잔동",  "하나씩",  "우엉대",  "하면",  "여긴",  "주소로",  "서리태",  "이만원",  "오만원",  "사만원",  "구만원",  "육만원",  "황금향",  "유기농",  "여독은",  "서을시",  "노원",  "공릉동",  "배즙",  "지금은",  "전화를",  "나중에",  "마가루",  "배송해",  "송금할",  "조리읍",  "이렇게",  "오후쯤",  "남겨",  "나눔몰",  "하는데",  "안되어",  "주문좀",  "하려구",  "홍합살",  "한병씩",  "서울시",  "한반도",  "마분말",  "하였",  "도마토",  "오늘도",  "엄마닭",  "도천동",  "하려고",  "어제꺼",  "오늘꺼",  "주문은",  "박스씩",  "제전번",  "하나만",  "마분말",  "하였",  "주안동",  "인천",  "옥동",  "신북면",  "오봉로",  "이쁘게",  "주실수",  "이르면",  "송금은",  "임돠",  "명계수",  "지내재",  "오전중",  "반포동",  "연락이",  "추가건",  "인천시",  "오이로",  "금액과",  "오분도",  "왕특",  "설향",  "주말에",  "위주로",  "주소랑",  "모르고",  "하나더",  "추가로",  "강릉",  "명의로",  "주소를",  "조개살",  "오색미",  "서울숲",  "도봉구",  "소자좀",  "은혜를",  "유색미",  "황매실",  "주문시",  "도화동",  "제일로",  "소개로",  "마카롱",  "지역이",  "이번엔",  "공유를",  "반려견",  "하루가",  "안깐거",  "주문함",  "반지락",  "소금의",  "김장을",  "어젠",  "오는건",  "주셔서",  "오셔서",  "현관옆",  "주문한",  "마늘쫑",  "설악면",  "제꺼",  "엄마댁",  "연근차",  "연잎차",  "현관앞",  "하미과",  "구리시",  "마산",  "마산시",  "주문차",  "기억은",  "노랑거",  "설향을",  "안깐",  "전화로",  "한사람",  "양파즙",  "성남시",  "기존과",  "주문이",  "어쩌죠",  "한건데",  "안빠름",  "금액이",  "홍매실",  "나도",  "한번씩",  "오늘은",  "고마워",  "최고로",  "주문일",  "노루골",  "하눅",  "주신거",  "은행명",  "지리산",  "국수집",  "안깐건",  "배송료",  "이용에",  "문의해",  "주시기",  "조금전",  "방법을",  "도움이",  "문구도",  "원주시",  "연수구",  "왕특특",  "마치고",  "마포구",  "인연에",  "신선한",  "주문건",  "선할인",  "서초동",  "안깐것",  "현관에",  "송파구",  "제이름",
    "이사님",  "천안",  "안산시",  "주셔도",  "방금전",  "기회가",  "설향도",  "위주소",  "신림동",  "연락을",  "한곳은",  "한세트",  "안되면",  "하면서",  "전화해",  "지곶동",  "도착전",  "하시고",  "제주도",  "공복에",  "마셔도",  "무리가",  "안가고",  "이쁘고",  "문제로",  "송금함",  "이전에",  "주셔유",  "금천구",  "이번에",  "구매로",  "신청은",  "주세용",  "소개로",  "이분은",  "오창읍",  "이름",  "안들어",  "배송은",  "강남콩",  "김포시",  "홍천읍",  "나올때",  "공지",  "박사님",  "목사님",  "신부님",  "여름을",  "가을을",  "정백당",  "배송처",  "이케아",  "연수로",  "전중",  "모바일",  "제외한",  "금액에",  "허용한",  "배려해",  "선결제",  "선결재",  "오리알",  "정확한",  "유정란",  "정도나",  "고객사",  "이메일",  "도령모",  "안감은",  "반반씩",  "배송중",  "홍어회",  "모듬쌈",  "심사역",  "마음이",  "지주소",  "제것도",  "문에",  "김장은",  "주문및",  "천일염",  "고추를",  "신청할",  "마시고",  "소개건",  "도착일",  "조카가",  "오기가",  "이거임",  "어디로",  "여기로",  "선배님",  "안넣고",  "안떠서",  "선입금",  "진젓",  "안양시",  "양주시",  "고암길",  "원장님",  "현미쌀",  "편한세상",  "김소양",  "신도림",  "남해덕성호",  "주문",  "부재시",  "부재중",  "신동아",  "전라남도",  "경상남도",  "경상북도",  "강원도",  "충청북도",  "충청남도",  "전라북도",  "경기도",  "고객님",  "차량은",  "차량의",  "추운데",  "소장님",  "이내로",  "소화가",  "백미쌀",  "성원이",  "서울에",  "원마을",  "인하로",  "성포동",  "오겹",  "제비앙",  "오리진",  "기기",  "이름",  "수령자",  "자정보",  "가정용",  "대장길",  "수령지",  "마을",  "아파트",  "이사장",  "한의원",  "경기도",  "전남",  "우리집",  "한박스",  "양구",  "유선상",  "한라봉",  "천혜양",  "천혜향",  "배송비",  "배송지",  "이킬로",  "이키로",  "오킬로",  "오키로",  "주세요",  "한의원",  "현금영수증",  "송도문화로",  "구매자",  "판매자",  "입금자",  "신세계",  "소나무",  "문앞에",  "나눠서",  "주시면",  "주문서",  "금액은",  "우엉차",  "이체한",  "강사님",  "구분만",  "마늘도",  "여기도",  "편안한",  "한다고",  "마상품",  "한병",  "여주즙",  "소포장",  "여기로",  "하느라",  "위와",  "이름은",  "금욜",  "문자로",  "한다",  "안함",  "여름만",  "이쁜",  "신주소",  "나무문",  "성내동",  "주시고",  "백도",  "황도",  "배송전",  "배송후",  "조끔찍",  "오후에",  "오전에",  "현영",  "문앞",  "주소지",  "제주시",  "천곡동",  "한상자",  "주소가",  "지켜봐",  "우리의",  "이름이",  "최선을",  "심의필",  "이것도",  "표면이",  "문의를",  "한달이",  "문제가",  "도착할",  "어제거",  "선물로",  "배송시",  "표를",  "선물용",  "이번주",  "편의점",  "고잔동",  "하나씩",  "우엉대",  "하면",  "여긴",  "주소로",  "서리태",  "이만원",  "오만원",  "사만원",  "구만원",  "육만원",  "황금향",  "유기농",  "여독은",  "서을시",  "노원",  "공릉동",  "배즙",  "지금은",  "전화를",  "나중에",  "마가루",  "배송해",  "송금할",  "조리읍",  "이렇게",  "오후쯤",  "남겨",  "나눔몰",  "하는데",  "안되어",  "주문좀",  "하려구",  "홍합살",
    "한병씩",  "서울시",  "한반도",  "마분말",  "하였",  "도마토",  "오늘도",  "엄마닭",  "도천동",  "하려고",  "어제꺼",  "오늘꺼",  "주문은",  "박스씩",  "제전번",  "하나만",  "마분말",  "하였",  "주안동",  "인천",  "옥동",  "신북면",  "오봉로",  "이쁘게",  "주실수",  "이르면",  "송금은",  "임돠",  "명계수",  "지내재",  "오전중",  "반포동",  "연락이",  "추가건",  "인천시",  "오이로",  "금액과",  "오분도",  "왕특",  "설향",  "주말에",  "위주로",  "주소랑",  "모르고",  "하나더",  "추가로",  "강릉",  "명의로",  "주소를",  "조개살",  "오색미",  "서울숲",  "도봉구",  "소자좀",  "은혜를",  "유색미",  "황매실",  "주문시",  "도화동",  "제일로",  "소개로",  "마카롱",  "지역이",  "이번엔",  "공유를",  "반려견",  "하루가",  "안깐거",  "주문함",  "반지락",  "소금의",  "김장을",  "어젠",  "오는건",  "주셔서",  "오셔서",  "현관옆",  "주문한",  "마늘쫑",  "설악면",  "제꺼",  "엄마댁",  "연근차",  "연잎차",  "현관앞",  "하미과",  "구리시",  "마산",  "마산시",  "주문차",  "기억은",  "노랑거",  "설향을",  "안깐",  "전화로",  "한사람",  "양파즙",  "성남시",  "기존과",  "주문이",  "어쩌죠",  "한건데",  "안빠름",  "금액이",  "홍매실",  "나도",  "한번씩",  "오늘은",  "고마워",  "최고로",  "주문일",  "노루골",  "하눅",  "주신거",  "은행명",  "지리산",  "국수집",  "안깐건",  "배송료",  "이용에",  "문의해",  "주시기",  "조금전",  "방법을",  "도움이",  "문구도",  "원주시",  "연수구",  "왕특특",  "마치고",  "마포구",  "인연에",  "신선한",  "주문건",  "선할인",  "서초동",  "안깐것",  "현관에",  "송파구",  "제이름",  "이사님",  "천안",  "안산시",  "주셔도",  "방금전",  "기회가",  "설향도",  "위주소",  "신림동",  "연락을",  "한곳은",  "한세트",  "안되면",  "하면서",  "전화해",  "지곶동",  "도착전",  "하시고",  "제주도",  "공복에",  "마셔도",  "무리가",  "안가고",  "이쁘고",  "문제로",  "송금함",  "이전에",  "주셔유",  "금천구",  "이번에",  "구매로",  "신청은",  "주세용",  "소개로",  "이분은",  "오창읍",  "이름",  "안들어",  "배송은",  "강남콩",  "김포시",  "홍천읍",  "나올때",  "공지",  "박사님",  "목사님",  "신부님",  "여름을",  "가을을",  "정백당",  "배송처",  "이케아",  "연수로",  "전중",  "모바일",  "제외한",  "금액에",  "허용한",  "배려해",  "선결제",  "선결재",  "오리알",  "정확한",  "유정란",  "정도나",  "고객사",  "이메일",  "도령모",  "안감은",  "반반씩",  "배송중",  "홍어회",  "모듬쌈",  "심사역",  "마음이",  "지주소",  "제것도",  "문에",  "김장은",  "주문및",  "천일염",  "고추를",  "신청할",  "마시고",  "소개건",  "도착일",  "조카가",  "오기가",  "이거임",  "어디로",  "여기로",  "선배님",  "안넣고",  "안떠서",  "선입금",  "진젓",  "안양시",  "양주시",  "고암길",  "원장님",  "현미쌀",  "편한세상",  "김소양",  "신도림",  "남해덕성호",  "주문",  "부재시",  "부재중",  "신동아",  "전라남도",  "경상남도",  "경상북도",  "강원도",  "충청북도",  "충청남도",  "전라북도",  "경기도",  "고객님",  "차량은",  "차량의",  "추운데",  "소장님",  "이내로",  "소화가",  "백미쌀",
    "성원이",  "서울에",  "원마을",  "인하로",  "성포동",  "오겹",  "제비앙",  "오리진",  "기기",  "이름",  "수령자",  "자정보",  "가정용",  "최상급",  "주소와",  "호수만",  "제수용", "안해도", "하세요", "수신자", "상도역",
    ]


# 불용어리스트를 정규표현식으로 바꿔주는 함수
def stopwordsforname_to_regex(stopwordsForName):
    regex_stopwordsforname = ""
    for i in range(len(stopwordsForName)):
        if stopwordsForName[i] != "":
            if regex_stopwordsforname == "":
                regex_stopwordsforname = stopwordsForName[i]
            else:
                regex_stopwordsforname = regex_stopwordsforname + "|" + stopwordsForName[i]
        else:
            pass
    return (regex_stopwordsforname)


# 메시지에서 이름 분리를 위한 불용어처리함수
def stopwords2(msg):
    regex = stopwordsforname_to_regex(stopwordsForName)
    stopwords = re.sub(regex, " ", msg)
    return stopwords


def getnameforcustomer(msg):
    regexForName = r"(김|이|박|최|정|강|조|윤|장|임|한|오|서|신|권|황|안|송|전|홍|류|고|문|양|손|배|조|백|허|유|남|심|노|정|하|곽|성|차|주|우|구|신|임|전|민|유|류|나|진|지|엄|채|원|천|방|공|강|현|함|변|염|양|변|여|추|노|도|소|신|석|선|설|마|길|주|연|방|위|표|명|기|반|라|왕|금|옥|육|인|맹|제|모|장|남궁|탁|국|여|진|어|은|편|가|간|갈|감|강|개|견|경|계|고|곡|공|곽|교|구|국|군|궁|궉|권|근|금|기|길|김|나|남|남궁|낭|내|노|뇌|누|단|담|당|대|도|독고|돈|동|동방|두|라|류|로|뢰|마|만|망절|매|맹|명|모|묘|목|묵|문|미|민|박|반|방|배|백|범|변|복|봉|부|빈|빙|사|사공|삼|상|서|서문|석|선|선우|설|섭|성|소|손|송|수|순|승|시|신|심|십|아|안|애|야|양|어|어금|엄|여|연|염|영|예|오|옥|온|옹|왕|요|용|우|운|원|위|유|육|윤|은|음|이|인|임|자|장|전|점|정|제|제갈|조|종|좌|주|준|즙|지|진|차|창|채|척|천|초|총|최|추|탁|탄|태|판|팽|편|평|포|표|풍|피|필|하|학|한|함|해|허|현|형|호|홍|화|환|황|황목|황보)([가-힣]{2})"
    namelist = re.findall(regexForName, msg)
    # print(f"fir_name_list : {namelist}")
    result = []
    for i in range(len(namelist)):
        result.append(namelist[i][0:3])
    # print(f"fir_result : {result}")
    return (result)


def getnamelist(msg):
    regexForName = r"(\s|^|/)(김|이|박|최|정|강|조|윤|장|임|한|오|서|신|권|황|안|송|전|홍|류|고|문|양|손|배|조|백|허|유|남|심|노|정|하|곽|성|차|주|우|구|신|임|전|민|유|류|나|진|지|엄|채|원|천|방|공|강|현|함|변|염|양|변|여|추|노|도|소|신|석|선|설|마|길|주|연|방|위|표|명|기|반|라|왕|금|옥|육|인|맹|제|모|장|남궁|탁|국|여|진|어|은|편|가|간|갈|감|강|개|견|경|계|고|곡|공|곽|교|구|국|군|궁|궉|권|근|금|기|길|김|나|남|남궁|낭|내|노|뇌|누|단|담|당|대|도|독고|돈|동|동방|두|라|류|로|뢰|마|만|망절|매|맹|명|모|묘|목|묵|문|미|민|박|반|방|배|백|범|변|복|봉|부|빈|빙|사|사공|삼|상|서|서문|석|선|선우|설|섭|성|소|손|송|수|순|승|시|신|심|십|아|안|애|야|양|어|어금|엄|여|연|염|영|예|오|옥|온|옹|왕|요|용|우|운|원|위|유|육|윤|은|음|이|인|임|자|장|전|점|정|제|제갈|조|종|좌|주|준|즙|지|진|차|창|채|척|천|초|총|최|추|탁|탄|태|판|팽|편|평|포|표|풍|피|필|하|학|한|함|해|허|현|형|호|홍|화|환|황|황목|황보)([가-힣]{1,2})(?=(로|으로|0|/|\s|입니다|사장|사장님|대표|대표님|대리|대리님|과장|과장님|실장|실장님|목사|목사님|차장|차장님|부장|부장님|이사|이사님|상무|상무님|전무|전무님|박사|박사님|교수|교수님|씨|님|$))"
    namelist = re.findall(regexForName, msg)
    # print(f"sec_name_list : {namelist}")
    result = []
    for i in range(len(namelist)):
        result.append(namelist[i][1:3])

    # print(f"sec_result : {result}")
    return (result)


# 상세주소 끝부분
def stopdetailaddr(msg):
    regex = "([A-Ca-c1-3]{1,1}|\d{1,4})(동|호|-|ㅡ)?"
    stopdetailaddr = re.sub(regex, " ", msg)
    return stopdetailaddr


# 일반주소 끝부분
def stopaddr(msg):
    regex = "[가-힣]{1,3}(특별자치시|광역시|특별시|시|군|구)?\s?[가-힣]{1,4}(읍|면|동|구|로)"
    stopaddr = re.sub(regex, " ", msg)
    return stopaddr


def msg_preprocessing(msg):
    items = re.findall('\(([^)]+)', msg)
    # print(items)
    num = 1
    fakemsg = ""
    if items != []:
        for item in items:
            if item in msg:
                if len(item) >= len(str(num)):
                    fakemsg = msg.replace("(" + item + ")", "(" + " " * (len(item) - len(str(num)) + 1) + ")")
                    num = num + 1
    else:
        fakemsg = msg


def nuguri_name(msg):
    msg = stopwords2(msg)
    msg = stopdetailaddr(msg)
    msg = stopaddr(msg)
    msg = msg.replace(".", " ")
    msg = msg.replace(",", " ")

    f = open("/content/drive/MyDrive/aimpact/nameDB.txt", 'r', encoding='UTF8')
    lines = f.readlines()
    submsg = msg
    submsg.replace("(", " ")
    submsg.replace(")", " ")
    submsg = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', submsg)
    submsg = re.sub(stopwordsforname_to_regex(stopwordsForName), ' ', submsg)

    namelist = getnamelist(submsg)
    trueNamelist = []
    for nameIndex in range(len(namelist)):
        for line in lines:
            if str(line.strip()) == namelist[nameIndex][1]:
                trueNamelist.append(namelist[nameIndex])
                break
            else:
                pass
    Namelist = []

    for NameIndex in range(len(trueNamelist)):
        Name = trueNamelist[NameIndex][0] + trueNamelist[NameIndex][1]
        Namelist.append(Name)

    return Namelist

In [47]:
# 이름 추출
def get_ML_name(sentence):
    name_list = []
    outputs = get_output(sentence)   # input_type = str    # outputs type = list
    # print(f"name_outputs : {outputs}")
    for j in range(1,len(outputs)):

        nes = outputs[j]['ne']
        if len(nes) != 0:

            for ne in nes:
                ne_label = ne['label']
                ne_form = ne['form']

                if ne_label == 'PS_NAME':
                    name = ne_form
                    name_list.append(name)

    return name_list

In [48]:
import itertools

def new_launch(raw_sentence):       # input = str / output = json
    
    # 전화번호 추출
    # print(f"raw_sentence : {raw_sentence}")
    phone_list, msg_list, raw_msg = getphonelist(raw_sentence)   # input = str / output = list, list, str
    # print(f"phone_list : {phone_list}")
    # print(f"msg_list : {msg_list}")
    # print(f"raw_msg : {raw_msg}")

    # 불용어 처리할 때 여기에 들어가야할 듯
    ## 일단 get_output 타기 전에 들어가야하는데,
    prepro_msg_list = preprocessing(msg_list)
    print(f"prepro_msg_list : {prepro_msg_list}")

    jusos, replace_msg = new_get_juso(prepro_msg_list, raw_msg)     # input = list, str / output = json, list
    # print(f"jusos : {jusos}")
    # print(f"replace_msg : {replace_msg}")

    # # name_list = []
    # # nuguri_name_list = []
    # ML_names= []
    # ML_name_list = []
    
    # #### 이름 확정 로직 수정 필요


    # for i in range(len(replace_msg)):
    #     # name = nuguri_name(replace_msg[i])
    #     # print(f"name : {name}")
    #     # nuguri_name_list.append(name)
    #     # name_list = list(itertools.chain(*nuguri_name_list))

    #     ML_name = get_ML_name(replace_msg[i])
    #     # print(f"ML_name : {ML_name}")
    #     ML_name_list.append(ML_name)
    #     # print(f"ML_name_list : {ML_name_list}")
    #     ML_names = list(itertools.chain(*ML_name_list))



    name_list = []
    nuguri_name_list = []
    ML_names= []
    ML_name_list = []
    
    #### 이름 확정 로직 수정 필요


    for i in range(len(replace_msg)):
        name = nuguri_name(replace_msg[i])
        nuguri_name_list.append(name)
        name_list = list(itertools.chain(*nuguri_name_list))

        ML_name = get_ML_name(replace_msg[i])
        ML_name_list.append(ML_name)
        ML_names = list(itertools.chain(*ML_name_list))

    json = {
        "names" : ML_names,
        "phones" : phone_list,
        "addresses" : jusos,
        "nuguri" : name_list

    }

    # print(f"json : {json}")

    return json 

    # name_list = []
    # nuguri_name_list = []
    # ML_names= []
    # ML_name_list = []
    
    # #### 이름 확정 로직 수정 필요


    # for i in range(len(replace_msg)):
    #     name = nuguri_name(replace_msg[i])
    #     nuguri_name_list.append(name)
    #     name_list = list(itertools.chain(*nuguri_name_list))

    #     ML_name = get_ML_name(replace_msg[i])
    #     ML_name_list.append(ML_name)
    #     ML_names = list(itertools.chain(*ML_name_list))

    # print(f"name_list : {name_list}")
    # print(f"ML_names : {ML_names}")
    # ps_name_list = []
    # final_name_list = []
    # final2=[]
    
    # for name in name_list:
    #     print(f"name : {name}")
    #     if name in ML_names:
    #         final_name = name
    #         final_name_list.append(final_name)
    #         print(f"final_name_list : {final_name_list}")

    #     else:
    #         ps_name_list = get_ML_name(name)
    #         final2 = list(itertools.chain(*ps_name_list))
    #         print(f"final2 : {final2}")

    # final_name_list.extend(final2)


In [49]:
# 어레인지 자료에서 오류 발생여부 확인
import pandas as pd

aimpact_dir = '/content/drive/MyDrive/aimpact'
aimpact_name = 'message_only_220126.csv'
aimpact_fPath = os.path.join(aimpact_dir, aimpact_name)
message = pd.read_csv(aimpact_fPath)

target = message['message']

In [50]:
target = message['message']
check=target[5696 + 15] #10406 = 10429 #11662 = 11686 #11767 = 11792      #12054 = 12078      # 12301 = 12326    # 12430 = 12456  #12612 = 12638 #13373 = 13404 #13432 = 13462 #13721 = 1375388
# print(check)
new_launch(check)

prepro_msg_list : ['김기태배송지 주소(39351) 경상북도 구미시 임은5길 7 (임은동) (주)케이엔정보기술    ', '             ']
raw_msg : 김기태배송지 주소(39351) 경상북도 구미시 임은5길 7 (임은동) (주)케이엔정보기술연락처1010-4850-7896
msg : 김기태배송지 주소 39351  경상북도 구미시 임은5길 7  임은동   주 케이엔정보기술    
form_list : ['김기태', '지', '39351', '경상북도', '구미시', '임은5길 7', '임은동', '주', '케이엔정보기술']
tag_list : ['PS_NAME', '', 'QT_ADDRESS', 'LCP_PROVINCE', 'LCP_CITY', 'AF_ROAD', 'LCP_COUNTY', 'OGG_ECONOMY', 'OGG_ECONOMY']
start_idx_list : [3, 4, 5]
end_idx_list : [0, 0, 0]
msg :              
form_list : []
tag_list : []


{'addresses': [], 'names': [], 'nuguri': [], 'phones': ['010-4850-7896']}

In [51]:
test = ''' 보내는 사람 
이종호 010 8750 4436

받는사람 
종호셋째이모 010-3736-9078       전북 무주군 안성면 덕산리 137-3번지 
2개 

이다영 010-4675-7336  
김포기 장기동 초당마을 주공아파트 108동 1005호
1개

이주리 010-3997-9325 
서울 관악구 청림3길 10 서희스타힐스 814호 
1개 

이효정 010-3386-4538 
서울 강북구 미아동 삼각산아이원 106동 1101호
 1개 

종호막내이모 010-4167-9775 울산 울주군 온산읍 서영남2길4 (우리분식)  
1개 

종호고모 010-5383-4538 
서울 성북구 길음뉴타운 푸르지오 211동 404호 
1개 

김희영 010-4109-8838
 경기도 가평군 설악면 자잠로 87-12 삼성빌라e동302호
 1개

이종호 010-8750-4436 
서울 성북구 동선동3가 259-8 삼광빌딩 103호 
2개
'''
new_launch(test)

prepro_msg_list : [' 보내는 사람 \n이종호', '              \n\n받는사람 \n종호셋째이모', '         ', '            전북 무주군 안성면 덕산리 137-3번지 \n2개 \n\n이다영', '                \n김포기 장기동 초당마을 주공아파트 108동 1005호\n1개\n\n이주리', '               \n서울 관악구 청림3길 10 서희스타힐스 814호 \n1개 \n\n이효정', '               \n서울 강북구 미아동 삼각산아이원 106동 1101호\n 1개 \n\n종호막내이모', '               울산 울주군 온산읍 서영남2길4 (우리분식)  \n1개 \n\n종호고모', '               \n서울 성북구 길음뉴타운 푸르지오 211동 404호 \n1개 \n\n김희영', '              \n 경기도 가평군 설악면 자잠로 87-12 삼성빌라e동302호\n 1개\n\n이종호', '               \n서울 성북구 동선동3가 259-8 삼광빌딩 103호 \n2개\n']
raw_msg :  보내는 사람 
이종호 010 8750 4436

받는사람 
종호셋째이모 010-3736-9078       전북 무주군 안성면 덕산리 137-3번지 
2개 

이다영 010-4675-7336  
김포기 장기동 초당마을 주공아파트 108동 1005호
1개

이주리 010-3997-9325 
서울 관악구 청림3길 10 서희스타힐스 814호 
1개 

이효정 010-3386-4538 
서울 강북구 미아동 삼각산아이원 106동 1101호
 1개 

종호막내이모 010-4167-9775 울산 울주군 온산읍 서영남2길4 (우리분식)  
1개 

종호고모 010-5383-4538 
서울 성북구 길음뉴타운 푸르지오 211동 404호 
1개 

김희영 010-4109-8838
 경기도 가평군 설악면 자잠로 87-12 삼성빌라e동302호
 1개

이종호 010-8750-4

{'addresses': [{'addrRocation': '안성면 덕산리 137-3번지',
   'address': '안성면 덕산리 137-3번지',
   'detailAddress': '안성면 덕산리 137-3번지'},
  {'addrRocation': '장기동 초당마을 주공아파트 108동 1005호',
   'address': '장기동 초당마을 주공아파트 108동 1005호',
   'detailAddress': '장기동 초당마을 주공아파트 108동 1005호'},
  {'addrRocation': '관악구 청림3길 10',
   'address': '관악구 청림3길 10',
   'detailAddress': '관악구 청림3길 10'},
  {'addrRocation': '미아동 삼각산아이원 106동 1101호',
   'address': '미아동 삼각산아이원 106동 1101호',
   'detailAddress': '미아동 삼각산아이원 106동 1101호'},
  {'addrRocation': '성북구 길음뉴타운 푸르지오 211동 404호',
   'address': '성북구 길음뉴타운 푸르지오 211동 404호',
   'detailAddress': '성북구 길음뉴타운 푸르지오 211동 404호'},
  {'addrRocation': '설악면 자잠로 87-12',
   'address': '설악면 자잠로 87-12',
   'detailAddress': '설악면 자잠로 87-12'},
  {'addrRocation': '성북구 동선동3가 259-8',
   'address': '성북구 동선동3가 259-8',
   'detailAddress': '성북구 동선동3가 259-8'}],
 'names': ['이효정', '김희영', '이종호'],
 'nuguri': ['이다영', '이주리', '이효정', '김희영', '이종호'],
 'phones': ['010 8750 4436',
  '010-3736-9078',
  '010-4675-7336',
  '0

# 주소 확정 로직

## 시작부분 

- 시작을 확정짓는 로직
    - 주소시작으로 보는 패턴이 나오는 경우 주소시작으로 확정

- 주소 시작으로 보는 패턴
    - LCP_CAPITALCITY
        - LCP_COUNTY
        - AF_ROAD
        - QT_ADDRESS
        - LC_OTHERS
        - AF_BUILDING
    - LCP_PROVINCE
        - LCP_COUNTY
        - LCP_CITY
        - AF_ROAD
        - QT_ADDRESS
        - AF_TRANSPORT (평택시 → 택시)
    - LCP_CITY
        - LCP_COUNTY
        - LCP_CITY
        - LCP_OTHERS
        - LCG_MOUNTAIN
        - LCG_ISLAND
        - AF_ROAD
        - AF_BUILDING
        - QT_ADDRESS
        - QT_OTHERS
        - PT_TREE


## 끝부분

- 끝을 확정짓는 로직
    - 주소 끝으로 추정되는 로직에 걸러지면 잠재적 확정(중간에 다른 사례가 나올경우 갱신필요)
    - 이름, 연락처, 주소시작, 메시지 끝이 나올 경우 잠재적 주소끝을 확정시킴

## 세부 주소 확정

# 주소 추출

# 이름 추출

# 전화번호 추출

# launch

# 예시문장

# Aimpact 자료에 적용